In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime, timedelta, time

In [2]:
area_path = '../dataset/area_timing/'
path_to_facility= '../dataset/facility_timing/'

In [5]:
sat_name = '111603'

In [40]:
# Read visibility to facilites data
dfs = []
for path, subdirs, files in os.walk(path_to_facility):
    for name in files:
        if sat_name in name:
            facility_csv_path = os.path.join(path, name)
            df = pd.read_csv(facility_csv_path, index_col=0)
            df['Start Time (UTCG)'] = pd.to_datetime(df['Start Time (UTCG)'], format='%d %b %Y %H:%M:%S.%f')
            df['Stop Time (UTCG)'] = pd.to_datetime(df['Stop Time (UTCG)'], format='%d %b %Y %H:%M:%S.%f')
            df['facility'] = name.split('-To-')[0]
            df['sat_name'] = sat_name
            mask = (df['Start Time (UTCG)'].dt.time >= time(7,30,00)) & (df['Start Time (UTCG)'].dt.time <= time(19,30,00))
            df_working = df.loc[mask]
            dfs.append(df_working)

facilities_df = pd.concat(dfs)

In [41]:
# Read area data
dfs = []
for path, subdirs, files in os.walk(area_path):
    for name in files:
        if sat_name in name:
            area_csv_path = os.path.join(path, name)
            df = pd.read_csv(area_csv_path, index_col=0)
            df['Start Time (UTCG)'] = pd.to_datetime(df['Start Time (UTCG)'], format='%d %b %Y %H:%M:%S.%f')
            df['Stop Time (UTCG)'] = pd.to_datetime(df['Stop Time (UTCG)'], format='%d %b %Y %H:%M:%S.%f')
            df['sat_name'] = sat_name
            mask = (df['Start Time (UTCG)'].dt.time >= time(9,00,00)) & (df['Start Time (UTCG)'].dt.time <= time(18,00,00))
            df_working = df.loc[mask]
            dfs.append(df_working)

area_df = pd.concat(dfs)

In [42]:
area_df = area_df.sort_values(by=['Start Time (UTCG)'])

In [49]:
area_df.reset_index(inplace=True, drop=True)

In [43]:
facilities_df = facilities_df.sort_values(by=['Start Time (UTCG)'])

In [47]:
facilities_df.reset_index(inplace=True, drop=True)

In [48]:
facilities_df

,Start Time (UTCG),Stop Time (UTCG),Duration (sec),facility,sat_name
0,2027-06-01 08:05:08.597,2027-06-01 08:18:02.725,774.128,Magadan2,111603
1,2027-06-01 08:05:08.597,2027-06-01 08:18:02.725,774.128,Magadan1,111603
2,2027-06-01 08:07:37.688,2027-06-01 08:18:07.297,629.609,Anadyr2,111603
3,2027-06-01 08:07:37.688,2027-06-01 08:18:07.297,629.609,Anadyr1,111603
4,2027-06-01 08:11:48.758,2027-06-01 08:15:50.607,241.849,Irkutsk,111603
...,...,...,...,...,...
712,2027-06-13 18:39:55.361,2027-06-13 18:48:25.866,510.505,Murmansk1,111603
713,2027-06-13 18:50:23.092,2027-06-13 19:02:53.224,750.132,Anadyr1,111603
714,2027-06-13 18:50:23.092,2027-06-13 19:02:53.224,750.132,Anadyr2,111603
715,2027-06-13 18:53:54.606,2027-06-13 19:01:15.782,441.176,Magadan2,111603


In [50]:
area_df

,Start Time (UTCG),Stop Time (UTCG),Duration (sec),sat_name
0,2027-06-01 09:46:51.002,2027-06-01 09:52:36.364,345.363,111603
1,2027-06-01 09:52:37.524,2027-06-01 09:53:38.951,61.427,111603
2,2027-06-01 11:23:11.938,2027-06-01 11:29:33.792,381.854,111603
3,2027-06-01 13:00:45.544,2027-06-01 13:05:07.811,262.268,111603
4,2027-06-01 14:36:49.173,2027-06-01 14:41:00.961,251.788,111603
...,...,...,...,...
75,2027-06-13 12:11:33.065,2027-06-13 12:16:47.503,314.437,111603
76,2027-06-13 12:16:50.059,2027-06-13 12:17:47.806,57.747,111603
77,2027-06-13 12:17:51.974,2027-06-13 12:18:10.510,18.536,111603
78,2027-06-13 13:49:36.192,2027-06-13 13:53:45.861,249.668,111603


In [51]:
search = '2027-06-02 09:59:03.148'
facilities_df['Start Time (UTCG)'].searchsorted(search)

67

In [53]:
res = facilities_df.iloc[[facilities_df['Start Time (UTCG)'].searchsorted(search)]]

In [61]:
filtered_facilities_df = pd.DataFrame(columns=facilities_df.columns)

In [62]:
filtered_facilities_df = pd.concat([filtered_facilities_df, res], ignore_index=True)

In [63]:
filtered_facilities_df

,Start Time (UTCG),Stop Time (UTCG),Duration (sec),facility,sat_name
0,2027-06-02 10:04:12.870,2027-06-02 10:14:39.823,626.954,Murmansk2,111603
